# MCP - Advanced

*Code Examples*
https://anthropic.skilljar.com/model-context-protocol-advanced-topics/295172

## Sampling
- Sampling - allows a Server to access an LLM through connected MCP Client
- This shifts responsibility and cost/complexity of text gen from server to client (simpler option, no need to enable MCP Server to access Claude)
- In Tool function - include Context argument
- Need server MCP code tool to include Sampling call
- Need Client MCP code to include Callback function (& pass Sampling callback into Session object)

- Process:
1. LLM - MCP Client (within a Next JS server) Tool Call Request to MCP Server;
2. MCP Server tool runs (w external service)
3. Once complete - MCP Client summarises/consolidates results OR **SAMPLING**: MCP Server offloads LLM Call to MCP Client for summarisation/consolidation then report back

## Logging & Notifications
- Frequent logging, progress meters - inside Tool, Call
- Need server MCP code Tool to include Context argument and use context built-in methods 'info', 'warning', 'debug', 'error', 'report_progress' (progress bar)
- Need Client MCP code to include Callback function (& pass Logging callback into Session object)

## Roots
- Access users and Claude to access to specific files/folders; permissioning system; no need to hardcode full paths, can just use file names (no need to search entire filesystems)
- Contains 2 Tools - read_dir (read list of files/folders in specific dir) & list_roots (list of roots - files/folderst the user has granted permissions to be read by server)
- You can provide Roots to Claude through tool or just inject them into prompt
- MCP Server - needs Files/Folders that other tools try to access are contained within a root (using Context and roots/filesystem built-in methods), and List_roots Tool (which accesses roots from Context), and is_path_allowed() check (which is then used throughouht when trying to use other tools)
- Main.py - contain root paths/directories from CLI arguments (optional)
- MCP Client - create roots and roots callbacks

## Transports and Communication (MCP)
- Messages Format (MCP server/client), STDIO transport, StreamableHTTP transport (& limitations)
- Many methods - Transport (Client/server exchange of JSON messages) - HTTP, STIO, Websockets...etc


### Messages ###
- Messages (client/server) - JSON format - MCP Spec (JSON or Typescript) -> https://github.com/modelcontextprotocol/modelcontextprotocol/tree/main/schema
- Different types - Some Client and some Server requests; List vs Notification
- Result Messages [need response] - Call Tool Request/Result, List Prompts Request/Result, Read Resource Request/Result, Initialise Request/Result, Create Message Result/Request, List Roots Result/Request
- Notification Messages [no response] - Progress Notification, Logging Message Notification, Tool List Changed Notification, Resource Updated Notification
- Some messages sent from Client to Server; some from Server to Client (ServerRequest, ServerNotification...etc)

### STDIO ###
- Easiest, simplest, both client/server same machine
1. Client launches MCP Server as subprocess
2. Client sends messages to MCP Server using server 'stdin' (whenever running a program in terminal - if we write - we're writing to stdin)
3. Server responds by writing to 'stdout'
- Critical - either Server or Client can send a message at any time
- Only appropriate - Client & Server in same machine
- Process: Initialize Request, Initialize Result, Initialize Notification; Logging, Tool Calls...etc
- 4 possible message types:
1. Initial request Client - Server (write to stdin)
2. Response Server - Client (write to stdout)
3. Initial request Server - Client (write to stdout)
4. Response Client - Server (write to stdin)
- Great advantage STDIO - Both Client AND Server can initiate request/send messages (Must be in Same Machine) -> This isn't the case for StreamableHTTP Transport (Can be in Separate Machines).

### StreamableHTTP Transport ###
- Client/server can be in different machines - Remote connection/hosting
- HTTP connection based
- More complex vs STDIO, harder to implement all 4 communication patterns - challenging to set up: Server initiate/send communication; specific MCP settings - big impact - stateless_http (True/False), json_response (True/False)
- HTTP (in general) - clients can easily Initiate Request to server (known URL) and Server can easily Respond to Client. But If Server wants to initiate request - it doesn't know HTTP Client address - challenging for Server to Initialise request.
- In MCP - HTTP 'traditional' - struggle to implement MCP Server 'Initiated' messages (Requests/Notifications): Create Message Request, List Roots Requests, Progress Notification, Logging Notification.
- Solution: StreamableHTTP - allows Server to make request to Client. However: using stateless_http and json_response to True will break the workaround.
- StreamableHTTP process:
1. MCP Client initialised connection (initialise request, initialise result, initalised notification) - includes Header - `mcp-session-id`
2. Client GET MCP server with Session ID -> opens SSE Server Sent Events connection - long-lived connection (for any future Server to Client requests e.g. Sampling [calls to LLM], Progress Notifications)
3. SSE Response allows Server to send requests to the client (i.e. Server initialised)
4. Client Requests - include Session ID as header; Server opens new SSE Response
5. First SSE Response - Server requests to client - long-lived (for any Server requests); Second/new SSE responses - closed after response provided (e.g. Tool call request)
6. New Messages: Progress Notification - using first SSE Response (long-lived), Logging Message/Tool Call Result - Second temp SSE Response (closed after Tool Call result).

- **State and StreamableHTTP Transport**
- If `stateless_http = True` or `json_response = True` - breaks StreamableHTTP Server to Client SSE (server-sent events) response abilities
- `stateless_http = True` - For very popular MCP Servers - horizontal scaling - Load balancer before MCP Server;
- If we need to build a scalable MCP Server - stateless_http = True; Clients don't get Session ID
- Disables: Server to Client requests, Sampling, Progress reports notifications, Subscriptions (resource updates), POST request responses aren't streamed (`json_response = True`)
- No Session ID = Can't use GET SSE Response pathway (can't use responses, sampling, progress logging, subscriptions)
- In Stateless Mode - Client initialisation (initial request+response/initialized notification) - not needed
- POST request responses aren't streamed if `json_response = True` - Request responses from Server to Client (no logging/streaming, just final results). You want simpler HTTP responses without streaming, just getting the final result as plain JSON.




